In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('your_file.csv')

In [6]:
df['diseases'] = df[['Angina', 'Anxiety disorder', 'Arthritis', 'Asthma',
                     'Atrial fibrillation/flutter', 'Cancer within 5 years',
                     'Chronic kidney disease(eGFR<60)', 'COPD',
                     'Coronary artery disease', 'Degenerative spine disease',
                     'Dementia', 'Depression', 'Diabetes',
                     'Fall within the past year', 'Heart Failure',
                     'Hypertension', 'Myocardial infarction',
                     'Peripheral vascular disease', 'Sensory impairment',
                     'Stroke/TIA', 'Use of 5 prescription drug']].sum(axis=1)

df['grip'] = df[['Grip Strength_1', 'Grip Strength_2']].max(axis=1)

In [7]:
df.rename(columns={'Frailty Index' : 'frailty_index', 
                   '순응도(%)' : '순응도',
                   'index date' : 'index_date',
                   '대상자이니셜' : '이니셜',
                   'total_score' : "SPPB",
                   'BIA_BMI' : 'BMI',
                   'Balance_1' : '일반자세', 
                   'Balance_2' : '반일렬자세', 
                   'Balance_3' : '일렬자세', 
                   'Gait speed (m/s)' : '평균보행속도', 
                   'Chair_sec' : '일어서기소요시간'                 
                   
                   }, inplace=True)

In [9]:
# Constants for readability and maintainability
GRIP_THRESHOLD_MALE = 32
GRIP_LOW_MALE = 26
GRIP_THRESHOLD_FEMALE = 20
GRIP_LOW_FEMALE = 16

CHAIR_THRESHOLD_HIGH = 61
CHAIR_THRESHOLD_MEDIUM = 16.7
CHAIR_THRESHOLD_LOW = 13.7
CHAIR_MINIMUM = 11.2

GAIT_THRESHOLD = 1
GAIT_LOW = 0.8
GAIT_LOWER = 0.6

# Improved function names and simplified logic

def minicog_score(x):
    if x < 0:
        return None
    if x == 5:
        return 0
    if x == 4:
        return 0.3
    if 1 <= x <= 3:
        return 0.7
    return 1

def grip_strength_score(data):
    grip = data.get('grip')
    sex = data.get('sex')
    
    if grip is None or grip < 0 or sex not in [1, 2]:
        return None

    if sex == 1:  # Male
        if grip >= GRIP_THRESHOLD_MALE:
            return 0
        if GRIP_LOW_MALE <= grip < GRIP_THRESHOLD_MALE:
            return 0.5
    else:  # Female
        if grip >= GRIP_THRESHOLD_FEMALE:
            return 0
        if GRIP_LOW_FEMALE <= grip < GRIP_THRESHOLD_FEMALE:
            return 0.5
    return 1

def chair_stand_score(x):
    if x < 0:
        return None
    if x >= CHAIR_THRESHOLD_HIGH:
        return 1
    if CHAIR_THRESHOLD_MEDIUM <= x < CHAIR_THRESHOLD_HIGH:
        return 0.75
    if CHAIR_THRESHOLD_LOW <= x < CHAIR_THRESHOLD_MEDIUM:
        return 0.5
    if CHAIR_MINIMUM <= x < CHAIR_THRESHOLD_LOW:
        return 0.25
    return 0

def gait_speed_score(x):
    if x < 0:
        return None
    if x >= GAIT_THRESHOLD:
        return 0
    if GAIT_LOW <= x < GAIT_THRESHOLD:
        return 0.3
    if GAIT_LOWER <= x < GAIT_LOW:
        return 0.7
    return 1

def nutrition_score(x):
    return 0 if x > 0 else 1 if x == 0 else None

In [12]:
df['minicog_score'] = df['Mini-cog'].apply(minicog_score)
df['gait_score'] = df['평균보행속도'].apply(gait_speed_score)
df['chair_score'] = df['일어서기소요시간'].apply(chair_stand_score)
df['grip_score'] = df.apply(grip_strength_score, axis = 1)
df['nutrition1_score'] = df['Nutritional status_1'].apply(nutrition_score)
df['nutrition2_score'] = df['Nutritional status_2'].apply(nutrition_score)
df['nutrition3_score'] = df['Nutritional status_3'].apply(nutrition_score)

In [22]:
columns_for_calculate = ['Angina', 'Anxiety disorder', 'Arthritis', 'Asthma',
       'Atrial fibrillation/flutter', 'Cancer within 5 years',
       'Chronic kidney disease(eGFR<60)', 'COPD', 'Coronary artery disease',
       'Degenerative spine disease', 'Dementia', 'Depression', 'Diabetes',
       'Fall within the past year', 'Heart Failure', 'Hypertension',
       'Myocardial infarction', 'Peripheral vascular disease',
       'Sensory impairment', 'Stroke/TIA', 'Use of 5 prescription drug',
       'ADL_1', 'ADL_2', 'ADL_3', 'ADL_4', 'ADL_5', 'ADL_6', 'ADL_7',  'IADL_1', 'IADL_2', 'IADL_3', 'IADL_4', 'IADL_5', 'IADL_6',
       'IADL_7', 'Nagi_1', 'Nagi_2', 'Nagi_3', 'Nagi_4',
       'Nagi_5', 'Rosow_Rreslau_1', 'Rosow_Rreslau_2',
       'Rosow_Rreslau_3', 'minicog_score', 'gait_score', 'chair_score', 'grip_score', 'nutrition1_score', 'nutrition2_score', 'nutrition3_score']

scoring_columns = ['diseases', 'ADL_total', 'IADL_total', 'Rosow_Rreslau_total', 'Nagi_total', 'nutrition1_score', 'nutrition2_score', 'nutrition3_score', 'minicog_score', 'gait_score', 'chair_score', 'grip_score']

In [16]:
df['missing'] = df[columns_for_calculate].isnull().sum(axis=1)
df['분모'] = 50 - df['missing']
df['score'] = df[scoring_columns].sum(axis=1)
df['CGA_FI'] = round(df['score'] / df['분모'], 3)

In [24]:
result_df = df[['연구ID', 'CGA_FI']]
result_df

,연구ID,CGA_FI
0,dy0001,0.006
1,dy0002,0.000
2,dy0003,0.047
3,dy0004,0.016
4,dy0005,0.000
...,...,...
270,sev0135,0.084
271,sev0136,0.180
272,sev0137,0.072
273,sev0138,0.060


<hr>